In [1]:
try:
    sc.stop()
except:
    pass
from pyspark import SparkContext
from bigdl.util.common import *


# create sparkcontext with bigdl configuration
sc = SparkContext.getOrCreate(conf=create_spark_conf().setMaster("local[*]"))
from pyspark.sql import SparkSession
spark = SparkSession(sparkContext = sc)
spark.conf.set("spark.sql.session.timeZone", "UTC")
init_engine()

Prepending /home/dyllanjr/anaconda3/envs/solarenv/lib/python3.6/site-packages/bigdl/share/conf/spark-bigdl.conf to sys.path


In [2]:
import numpy as np

npzfile = np.load('/home/dyllanjr/Solar_Irradiance_Prediction/data/processed/Arkansas_look_back_48.npz')

In [3]:
npzfile.files

['x', 'y']

In [4]:
Xs = npzfile['x']
y = npzfile['y']

['Cloud Type',
 'Dew Point',
 'Fill Flag',
 'Wind Speed',
 'Surface Albedo',
 'Temperature',
 'Solar Zenith Angle',
 'Wind Direction']

In [6]:
### I need to split my Xs into categorical and numericals
#I'm going to just use the numerical columns first.
cat_data = Xs[:,:,[0,2]]
num_data = Xs[:,:,[1,3,4,5,6,7]]

In [63]:
cat_data[0]

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [4., 0.],
       [4., 0.],
       [0., 0.],
       [4., 4.],
       [4., 0.],
       [4., 0.]])

In [61]:
y.shape

(7299, 1)

In [8]:
sum(y[0])

array([3211], dtype=int32)

In [9]:
#I just want to predict the sum of GHI
y = np.sum(y, axis=1)

In [10]:
y[0]

array([3211])

In [11]:
7299*.8

5839.200000000001

In [67]:
cat_data_train = cat_data[:5840]
cat_data_test = cat_data[5840:]
num_data_train = num_data[:5840]
num_data_test = num_data[5840:]
X_train = Xs[:5840]
X_test = Xs[5840:]
y_train = y[:5840]
y_test = y[5840:]
y_train = np.reshape(y_train, (y_train.shape[0], 1))
y_test = np.reshape(y_test, ( y_test.shape[0], 1))

In [23]:
print(Xs.shape, cat_data.shape, num_data.shape)

(7299, 48, 8) (7299, 48, 2) (7299, 48, 6)


In [24]:
print(num_data_train.shape, y_train.shape)

(5840, 48, 6) (5840, 1)


In [ ]:
cat

In [15]:
###Numerical model only architecture
#Adam optimizer
#Init learning rate .001
#max epochs 200
#300*3 hidden lstm units
#3 hidden layers
# from bigdl.nn.layer import Sequential, Recurrent, LSTM, InferReshape, Select, Linear
from bigdl.nn.keras.topology import Sequential
from bigdl.nn.keras.layer import LSTM, Dense

In [75]:
def build_model(inp_shape=(48,6), hidden_size = 150, out_size = 1):

    model = Sequential()
    model.add(LSTM(output_dim=hidden_size, input_shape=inp_shape, return_sequences = True))
    model.add(LSTM(output_dim=hidden_size, return_sequences = True))
    model.add(LSTM(output_dim=hidden_size))
    model.add(Dense(1))
    return model

model = build_model()

creating: createSequential


TypeError: __init__() got an unexpected keyword argument 'output_dim'

In [25]:
from bigdl.util.common import *
from bigdl.optim.optimizer import *
from bigdl.nn.criterion import *


In [21]:
spark.catalog.clearCache()

In [76]:
from bigdl.nn.layer import *

def build_model(input_size, hidden_size, output_size):
    model = Sequential()
    rec1 = Recurrent()
    rec1.add(LSTM(input_size, hidden_size))
    model.add(InferReshape([-1, input_size], True))
    model.add(rec1)
    model.add(Select(2, -1))
    model.add(Linear(hidden_size, output_size))
    return model
model = build_model(48, 300, 1)

creating: createSequential
creating: createRecurrent
creating: createTanh
creating: createSigmoid
creating: createLSTM
creating: createInferReshape
creating: createSelect
creating: createLinear


In [69]:
# def build_model(input_size, hidden_size, output_size):
#     model = Sequential()
#     rec1 = Recurrent()
#     rec1.add(LSTM(input_size, hidden_size))
#     model.add(InferReshape([-1, input_size], True))
#     model.add(rec1)
#     rec2 = Recurrent()
#     rec2.add(LSTM(hidden_size, hidden_size))
#     model.add(rec2)
#     rec3 = Recurrent()
#     rec3.add(LSTM(hidden_size, hidden_size))
#     model.add(rec3)
#     model.add(Select(2, -1))
#     model.add(Linear(hidden_size, output_size))
#     return model
# model = build_model(48, 300, 1)

creating: createSequential
creating: createRecurrent
creating: createTanh
creating: createSigmoid
creating: createLSTM
creating: createInferReshape
creating: createRecurrent
creating: createTanh
creating: createSigmoid
creating: createLSTM
creating: createRecurrent
creating: createTanh
creating: createSigmoid
creating: createLSTM
creating: createSelect
creating: createLinear


In [77]:
from bigdl.util.common import to_sample_rdd
train_data_num = to_sample_rdd(num_data_train, y_train)
test_data_num = to_sample_rdd(num_data_test, y_test)
train_data = to_sample_rdd(X_train, y_train)
test_data = to_sample_rdd(X_test, y_test)

In [78]:
show_bigdl_info_logs()
redire_spark_logs()

In [79]:
optimizer = Optimizer(
    model = model,
    training_rdd = train_data,
    criterion = MSECriterion(),
    optim_method = Adam(),
    end_trigger = MaxEpoch(200),
    batch_size=48)

optimizer.set_validation(
    batch_size=48,
    val_rdd=test_data,
    trigger=EveryEpoch(),
    val_method=[Loss(MSECriterion())]
)

optimizer.set_checkpoint(EveryEpoch(), '/home/dyllanjr/Solar_Irradiance_Prediction/models')

optimizer.optimize()
#This took 4.5 hours!
#I'm going to switch to Keras to finish the project.

creating: createMSECriterion
creating: createAdam
creating: createMaxEpoch
creating: createDistriOptimizer
creating: createEveryEpoch
creating: createMSECriterion
creating: createLoss


/home/dyllanjr/anaconda3/envs/solarenv/lib/python3.6/site-packages/bigdl/optim/optimizer.py:864: UserWarning: You are recommended to use `create` method to create an optimizer.
  warnings.warn("You are recommended to use `create` method to create an optimizer.")


creating: createEveryEpoch


In [80]:
test_results = model.evaluate(test_data_num, 18, [Loss(MSECriterion())])

creating: createMSECriterion
creating: createLoss


In [81]:
print(test_results[0])

Evaluated result: 32194012.0, total_num: 1459, method: Loss


In [52]:
np.sqrt(104151304)

10205.454619956918

In [82]:
np.sqrt(32194012)

5673.976735941028

### Note
#### Unfortunately, I couldn't get this one layer 300 hidden unit LSTM network to perform well. I suspect two issues:
- The framework of BigDL doesn't support Embbeding layers combined with LSTM layers
- The framework of Big DL doesn't support multiple LSTM layers
    - Missing return_sequences = True param in LSTM()

In [ ]:
def run_optimizer(model, epochs = 200, optim_method = Adam())
optimizer = Optimizer(
    model = model,
    training_rdd = train_data,
    criterion = MSECriterion(),
    optim_method = optim_method,
    end_trigger = MaxEpoch(epochs),
    batch_size=48)

optimizer.set_validation(
    batch_size=48,
    val_rdd=test_data,
    trigger=EveryEpoch(),
    val_method=[Loss(MSECriterion())]
)

optimizer.set_checkpoint(EveryEpoch(), '/home/dyllanjr/Solar_Irradiance_Prediction/models')

optimizer.optimize()